看一个用生成器来计算100以内斐波那契数列的例子。我们先用普通递归方式来进行计算。

In [1]:
a = b = 1
while a < 100:
    a, b = b, a + b
    print a,


1 2 3 5 8 13 21 34 55 89 144


再来用yield和生成器来计算斐波那契数列，该函数形成一个无限循环的生成器，由函数调用者显式地控制迭代次数。

In [2]:
def fibonacci():
    a = b = 1
    yield a
    yield b
    while True:
        a, b = b, a+b
        yield b

num = 0
fib = fibonacci()
while num < 100:
    num = next(fib)
    print num,


1 1 2 3 5 8 13 21 34 55 89 144


用yield实现协程调度的原理

我们现在利用yield关键字会自动冻结函数堆栈的特性，想象一下，假如现在有两个函数f1()和f2(),各自包含yield语句，见下例。主线程先启动f1(), 当f1()执行到yield的时候，暂时返回。这时主线程可以将执行权交给f2()，执行到f2()的yield后，可以再将执行权交给f1()，从而实现了在同一线程中交错执行f1()和f2()。f1()与f2()就是协同执行的程序，故名协程。

我们尝试用yield建立协程，来解决Producer-Consumer问题。

In [3]:
# -*- coding: utf-8 -*-
import random
 
def get_data():
    """返回0到9之间的3个随机数，模拟异步操作"""
    return random.sample(range(10), 3)
 
def consume():
    """显示每次传入的整数列表的动态平均值"""
    running_sum = 0
    data_items_seen = 0
    
    while True:
        print('Waiting to consume')
        data = yield
        data_items_seen += len(data)
        running_sum += sum(data)
        print('Consumed, the running average is {}'.format(running_sum / float(data_items_seen)))
 
def produce(consumer):
    """产生序列集合，传递给消费函数（consumer）"""
    while True:
        data = get_data()
        print('Produced {}'.format(data))
        consumer.send(data)
        yield
 
if __name__ == '__main__':
    consumer = consume()
    consumer.send(None) 
    producer = produce(consumer)
 
    for _ in range(10):
        print('Producing...')
        next(producer)

Waiting to consume
Producing...
Produced [3, 6, 8]
Consumed, the running average is 5.66666666667
Waiting to consume
Producing...
Produced [9, 2, 0]
Consumed, the running average is 4.66666666667
Waiting to consume
Producing...
Produced [7, 4, 1]
Consumed, the running average is 4.44444444444
Waiting to consume
Producing...
Produced [0, 5, 8]
Consumed, the running average is 4.41666666667
Waiting to consume
Producing...
Produced [8, 5, 0]
Consumed, the running average is 4.4
Waiting to consume
Producing...
Produced [6, 0, 3]
Consumed, the running average is 4.16666666667
Waiting to consume
Producing...
Produced [6, 3, 8]
Consumed, the running average is 4.38095238095
Waiting to consume
Producing...
Produced [4, 6, 0]
Consumed, the running average is 4.25
Waiting to consume
Producing...
Produced [9, 6, 3]
Consumed, the running average is 4.44444444444
Waiting to consume
Producing...
Produced [9, 0, 1]
Consumed, the running average is 4.33333333333
Waiting to consume


字符串的格式化处理

In [4]:
a = 'liu'
b = 'zhenya'
a+b

'liuzhenya'

In [5]:
'{},{}!'.format(a,b)

'liu,zhenya!'

In [8]:
'%s,%s!' % (a,b)   #二元运算符，前后都需要空格

'liu,zhenya!'

In [ ]:
#推荐使用 “with”语句 以管理文件:

with open("hello.txt") as hello_file:
    for line in hello_file:
        print line
#对于不支持使用”with”语句的类似文件的对象,使用 contextlib.closing():

import contextlib

with contextlib.closing(urllib.urlopen("http://www.python.org/")) as front_page:
    for line in front_page:
        print line

在Python中, pydoc以及单元测试要求模块必须是可导入的.
你的代码应该在执行主程序前总是检查 if __name__ == '__main__' , 这样当模块被导入时主程序就不会被执行.
def main():
      ...

if __name__ == '__main__':
    main()

# 2015-09-18 日报

模块codecs提供了一个open()方法，可以指定一个编码打开文件，使用这个方法打开的文件读取返回的将是unicode。写入时，如果参数是unicode，则使用open()时指定的编码进行编码后写入；如果是str，则先根据源代码文件声明的字符编码，解码成unicode后再进行前述操作。相对内置的open()来说，这个方法比较不容易在编码上出现问题。

In [ ]:
# coding: GBK

import codecs

f = codecs.open('test.txt', encoding='UTF-8')
u = f.read()
f.close()
print type(u) # <type 'unicode'>

f = codecs.open('test.txt', 'a', encoding='UTF-8')
# 写入unicode
f.write(u)

# 写入str，自动进行解码编码操作
# GBK编码的str
s = '汉'
print repr(s) # '\xba\xba'
# 这里会先将GBK编码的str解码为unicode再编码为UTF-8写入
f.write(s) 
f.close()

#Python日报 2015-09-20

In [1]:
###函数式编程
lst = [1,-1,3,-2]
lst2 = list()
for i in range(len(lst)): #模拟经典for循环
    if lst[i] > 0:
        lst2.append(lst[i])
lst2

[1, 3]

In [4]:
lst2 = filter(lambda x: x>0,lst) #通过filter完成复合要求的选取,把True的返回回来为一个list
lst2

[1, 3]

In [7]:
class greater_than_helper:
    def __init__(self, minval):
        self.minval = minval
    def is_greater_than(self, val):
        return val > self.minval

def my_filter(lst, minval):
    helper = greater_than_helper(minval)
    return filter(helper.is_greater_than, lst)

my_filter(lst,1)

[3]

python中set的作用是去除重复的元素

In [7]:
a = [1,1,2,3]
print a 
print type(a)
b = set(a)
print type(b)
b.add(2)

[1, 1, 2, 3]
<type 'list'>
<type 'set'>


#Python日报 2015-10-09

In [1]:
def argFunc(positional_arg, keyword_arg="foo", *tuple_grp_nonkw_args, **dict_grp_kw_args):
    print "positional_arg:", positional_arg
    print "keyword_arg:", keyword_arg
    for arg in tuple_grp_nonkw_args:
        print "additional non-keyword arg:", arg
    for argKey in dict_grp_kw_args.keys():
        print "additional keyword arg: {'%s': %s}" %(argKey, dict_grp_kw_args[argKey])
 
argFunc(3, 4, name="wilber", age=28)
print
argFunc(3, 4, "hello", "world", name="wilber", age=28)
print

positional_arg: 3
keyword_arg: 4
additional keyword arg: {'age': 28}
additional keyword arg: {'name': wilber}

positional_arg: 3
keyword_arg: 4
additional non-keyword arg: hello
additional non-keyword arg: world
additional keyword arg: {'age': 28}
additional keyword arg: {'name': wilber}



In [2]:
addNum = lambda x, y: x+y
print addNum
print "3 + 4 = ", addNum(3, 4)

<function <lambda> at 0x7fd7641db230>
3 + 4 =  7


# reduce()

reduce函数的完整形式为reduce(func, seq [, init])：func是一个二元函数；reduce对seq中的每一个元素进行迭代，每次迭代将上一次的迭代结果（第一次时使用init，如没有init，则使用seq的第一个元素）与下一个元素执行func函数。

看一个例子，通过reduce函数进行求和操作：

In [3]:
print reduce(lambda x, y: x + y, range(10)) 
print reduce(lambda x, y: x + y, range(10), 100)#100是初始值

45
145


In [4]:
import argparse
#http://www.blog.pythonlibrary.org/2015/10/08/a-intro-to-argparse/#网址介绍了如何通过commandline参数传递

In [5]:
a_list = ['a',2,[4,5]]
a_tuple = (2,4,(4,6))

In [9]:
a_tuple[1] =3#cannot be modified

TypeError: 'tuple' object does not support item assignment

In [11]:
def foo(x):
    if x < 0:
        print 'negative!'
    elif x == 0:
    # TODO: put something smart here
        pass # pass do nothing
    else:
        print 'positive!'
foo(1)

positive!


In [12]:
#改写多变量函数变成确定某些数值输入的情况
def add_numbers(x, y):
    return x + y
from functools import partial
add_five = partial(add_numbers, 5)
add_five(3)

8

In [ ]:
#在pandas中计算60天的平均
# compute 60-day moving average of time series x
ma60 = lambda x: pandas.rolling_mean(x, 60)
# Take the 60-day moving average of all time series in data
data.apply(ma60)

In [14]:
from IPython.display import Image
Image(url = "http://www.th7.cn/d/file/p/2012/11/19/6d0292169de4a0d163786413c1292721.png")

#dict的拷贝问题

In [2]:
first = {'key': 'value'}
second = dict(first)
second['key'] = 'other'
print first
print second

{'key': 'value'}
{'key': 'other'}


In [3]:
first = {'key': 'value'}
second = first
second['key'] = 'other'
print first
print second

{'key': 'other'}
{'key': 'other'}


In [6]:
first = {'key': ['value']}
second = dict(first)
second['key'].append('second value')
print first
print second

{'key': ['value', 'second value']}
{'key': ['value', 'second value']}


In [7]:
import copy
first = {'key': ['value']}
second = copy.deepcopy(first)
second['key'].append('second value')
print first
print second

{'key': ['value']}
{'key': ['value', 'second value']}


In [1]:
import numpy as np
np?

In [11]:
L3 = [True,"2",3.9,4]
[type(item) for item in L3]

[bool, str, float, int]

In [12]:
np.zeros(10,dtype=np.int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
np.ones((3,5))

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [14]:
np.full((2,3),3.2)

array([[ 3.2,  3.2,  3.2],
       [ 3.2,  3.2,  3.2]])

In [15]:
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [16]:
np.linspace(0, 1, 5)

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [17]:
# Create a (3 x 3) array of uniformly-distributed
# random values between 0 and 1
np.random.random((3, 3))

# Create a (3 x 3) array of normally-distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

# Create a (3 x 3) array of random integers in the interval
# [0, 10)
np.random.randint(0, 10, (3, 3))

# Create a (3 x 3) identity matrix
np.eye(3)


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [19]:
x = np.arange(10)

In [20]:
x[:5]

array([0, 1, 2, 3, 4])

In [23]:
x[::2]==x[0::2]

array([ True,  True,  True,  True,  True], dtype=bool)

In [6]:
x2 = np.random.random((3,4))
x2

array([[ 0.11414236,  0.86048556,  0.73307016,  0.19317354],
       [ 0.48667357,  0.13731799,  0.16488646,  0.37493435],
       [ 0.36448091,  0.4472653 ,  0.70460082,  0.62355654]])

In [8]:
x2[1:2,:]

array([[ 0.48667357,  0.13731799,  0.16488646,  0.37493435]])

In [9]:
x2[:3, ::2] # all rows, every other column

array([[ 0.11414236,  0.73307016],
       [ 0.48667357,  0.16488646],
       [ 0.36448091,  0.70460082]])

In [10]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

In [11]:
x = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x, [3, 5])
print(x1, x2, x3)

(array([1, 2, 3]), array([99, 99]), array([3, 2, 1]))


In [2]:
import pandas as pd
import numpy as np
data = pd.DataFrame(np.random.rand(3, 2),columns=['foo', 'bar'],index=['a', 'b', 'c'])
data

,foo,bar
a,0.423354,0.025779
b,0.412448,0.360707
c,0.172853,0.420018


In [3]:
data[data['foo']>0.4]

,foo,bar
a,0.423354,0.025779
b,0.412448,0.360707


In [4]:
data

,foo,bar
a,0.423354,0.025779
b,0.412448,0.360707
c,0.172853,0.420018


In [6]:
data.loc['a']# a行

foo    0.423354
bar    0.025779
Name: a, dtype: float64

In [8]:
data.iloc[1]# iloc是直接第几行

foo    0.412448
bar    0.360707
Name: b, dtype: float64

In [9]:
data.foo

a    0.423354
b    0.412448
c    0.172853
Name: foo, dtype: float64

In [10]:
data.ix[1]

foo    0.412448
bar    0.360707
Name: b, dtype: float64

In [12]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
print A
print B

0    2
1    4
2    6
dtype: int64
1    1
2    3
3    5
dtype: int64


In [13]:
A+B

0   NaN
1     5
2     9
3   NaN
dtype: float64

In [14]:
A.add(B,fill_value=0)

0    2
1    5
2    9
3    5
dtype: float64

In [15]:
data = pd.Series([1, np.nan, 'hello', None])

In [18]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [19]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [20]:
data.dropna()

0        1
2    hello
dtype: object

In [22]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a     1
b   NaN
c     2
d   NaN
e     3
dtype: float64

In [25]:
data.fillna(method='ffill')

a    1
b    1
c    2
d    2
e    3
dtype: float64

In [27]:
index = [('California', 2000), ('California', 2010),
('New York', 2000), ('New York', 2010),
('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
18976457, 19378102,
20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [28]:
mindex = pd.MultiIndex.from_tuples(index)

In [31]:
pop =pop.reindex(mindex)

In [32]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [38]:
pop = pop.unstack()
pop

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [36]:
pop.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [39]:
df = pd.DataFrame(np.random.rand(4, 2),
index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
columns=['data1', 'data2'])
df

data1     data2
a 1  0.434107  0.240539
  2  0.896764  0.568486
b 1  0.250230  0.708597
  2  0.194979  0.543152

In [40]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.743098
      2      0.082845
c     1      0.748928
      2      0.698362
b     1      0.915912
      2      0.702045
dtype: float64

In [43]:
data = data.sort_index()
data

char  int
a     1      0.743098
      2      0.082845
b     1      0.915912
      2      0.702045
c     1      0.748928
      2      0.698362
dtype: float64

In [47]:
data.index.names['char','int','value']

TypeError: list indices must be integers, not tuple

In [49]:
import pandas as pd
data = ['peter', 'Paul', 'MARY', 'gUIDO']
names = pd.Series(data)
names

0    peter
1     Paul
2     MARY
3    gUIDO
dtype: object

In [50]:
names.str.capitalize()

0    Peter
1     Paul
2     Mary
3    Guido
dtype: object

In [57]:
a="hello\n"

'hello\n'

In [59]:
a.replace('\n','')

'hello'

2015-10-28python RIBAO

In [3]:
strList = ["Python", "is", "good"]  
 
res =  ''.join(strList) #Python is good #list to str
res

'Python/is/good'

In [4]:
words="hello"
words[::-1]#reverse

'olleh'

In [5]:
#列表求和，最大值，最小值，乘积
numList = [1,2,3,4,5]   
 
sum = sum(numList)  #sum = 15
maxNum = max(numList) #maxNum = 5
minNum = min(numList) #minNum = 1
from operator import mul
prod = reduce(mul, numList, 1) #prod = 120 默认值传1以防空列表报错

In [6]:
keys = ['Name', 'Sex', 'Age']
values = ['Tim', 'Male', 23]
 
dic = dict(zip(keys, values))
dic

{'Age': 23, 'Name': 'Tim', 'Sex': 'Male'}

12月1日python日报

In [1]:
mynumber = 5

print "Python 2:"
print "The number is %d" % (mynumber)
print mynumber / 2,
print mynumber // 2

from __future__ import print_function
from __future__ import division

print("nPython 3:")
print("The number is {}".format(mynumber))
print(mynumber / 2, end=" ")
print(mynumber // 2)

Python 2:
The number is 5
2 2
nPython 3:
The number is 5
2.5 2


In [3]:
#链式比较操作符
mynumber = 3

if 4 > mynumber > 2:
    print("Chained comparison operators work! \n" * 3)

Chained comparison operators work! 
Chained comparison operators work! 
Chained comparison operators work! 



In [4]:
#collections.Counter
#Python的集合库看上去是最好的。在计算需要集合中元素的个数时，StackOverflow找到的答案是创建有序字典，但我坚持使用一个代码片段来创建字典，计算结果中元素出现的频率。直到有一天，我发现可以用collections.deque。


from collections import Counter
from random import randrange
import pprint

mycounter = Counter()

for i in range(100):
    random_number = randrange(10)
    mycounter[random_number] += 1

for i in range(10):
    print(i, mycounter)

0 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
1 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
2 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
3 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
4 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
5 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
6 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
7 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
8 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})
9 Counter({5: 15, 2: 13, 1: 12, 7: 11, 3: 10, 8: 10, 9: 10, 4: 8, 6: 6, 0: 5})


In [6]:
import subprocess
output = subprocess.check_output("ls", shell=True)
print(output)

anaconda
anyconnect-64.sh
a.py~
baidu.pac
catkin_ws
curve.txt~
Desktop
Documents
Downloads
dudrive-gazebo
eclipse
eigen
eigen.tar.gz
examples.desktop
get_map_center_line
git
learnpython
loganalysis
Music
path1.txt~
path2.txt~
Pictures
Public
pythonribao_daily.ipynb
result~
result2~
RosbagPandas
rosbag_python_analysis
ros_install.sh
samba_data
spirallines
Templates
Untitled Document~
Videos
workspace
workspace1
主要参考论文.pdf



2015年12月7日python日报

In [1]:
#装饰器用于在函数运行超时之后
import signal,functools #下面会用到的两个库 
class TimeoutError(Exception): pass #定义一个Exception，后面超时抛出 

def timeout(seconds, error_message = 'Function call timed out'):
  def decorated(func):
    def _handle_timeout(signum, frame):
      raise TimeoutError(error_message)
    def wrapper(*args, **kwargs):
      signal.signal(signal.SIGALRM, _handle_timeout)
      signal.alarm(seconds)
      try:
        result = func(*args, **kwargs)
      finally:
        signal.alarm(0)
      return result
    return functools.wraps(func)(wrapper)
  return decorated

@timeout(5) #限定下面的slowfunc函数如果在5s内不返回就强制抛TimeoutError Exception结束 
def slowfunc(sleep_time):
  import time
  time.sleep(sleep_time) #这个函数就是休眠sleep_time秒 

slowfunc(3) #sleep 3秒，正常返回 没有异常 


slowfunc(10) #被终止 

TimeoutError: Function call timed out

Trace函数

有时候出于演示目的或者调试目的，我们需要程序运行的时候打印出每一步的运行顺序 和调用逻辑。
类似写bash的时候的bash -x调试功能，然后Python解释器并没有 内置这个时分有用的功能，
那么我们就“自己动手，丰衣足食”。


In [3]:
import sys,os,linecache
def trace(f):
  def globaltrace(frame, why, arg):
    if why == "call": return localtrace
    return None
  def localtrace(frame, why, arg):
    if why == "line":
      # record the file name and line number of every trace 
      filename = frame.f_code.co_filename
      lineno = frame.f_lineno
      bname = os.path.basename(filename)
      print "{}({}): {}".format(  bname,
                    lineno,
                    linecache.getline(filename, lineno)),
    return localtrace
  def _f(*args, **kwds):
    sys.settrace(globaltrace)
    result = f(*args, **kwds)
    sys.settrace(None)
    return result
  return _f
#使用：
@trace
def xxx():
    print 1
    print 22
    print 333

xxx() #调用 

<ipython-input-3-cf2b7cb41734>(25):     print 1
iostream.py(208):         if self.pub_socket is None:
iostream.py(212):             if not isinstance(string, unicode_type):
iostream.py(213):                 string = string.decode(self.encoding, 'replace')
utf_8.py(16):     return codecs.utf_8_decode(input, errors, True)
iostream.py(215):             is_child = (self._check_mp_mode() == CHILD)
iostream.py(104):         if not self._pipe_flag or self._is_master_process():
iostream.py(94):         return os.getpid() == self._master_pid
iostream.py(105):             return MASTER
iostream.py(216):             self._buffer.write(string)
1iostream.py(217):             if is_child:
iostream.py(225):             current_time = time.time()
iostream.py(226):             if self._start < 0:
iostream.py(228):             elif current_time - self._start > self.flush_interval:
 iostream.py(208):         if self.pub_socket is None:
iostream.py(212):             if not isinstance(string, unicode_type)

python日报2015年12月8日

In [3]:
from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
display(text)

def handle_submit(sender):
    print (text.value)
text.on_submit(handle_submit)

In [4]:
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print ("Button clicked.")
button.on_click(on_button_clicked)

Button clicked.


In [7]:
def f(x):
    print(x)
widgets.interact(f,x=10)

5


In [8]:
widgets.interact(f,x=True)

True


In [9]:
widgets.interact(f,x="test")

a


In [ ]:
inputText的小写变成大写，放在outputText

In [11]:
outputText = widgets.Text()
outputText

In [12]:
inputText = widgets.Text()

def makeUpperCase(sender):
    outputText.value = inputText.value.upper()
inputText.on_submit(makeUpperCase)
inputText

In [4]:
from IPython.html.widgets import *
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

t = np.arange(0.0, 1.0, 0.01)

def pltsin(f):
    plt.plot(t,np.sin(2*np.pi*t*f))
    plt.show()
widgets.interact(pltsin,f=(1,10,0.1))

5.0


In [21]:
%matplotlib notebook
import pandas as pd  
import matplotlib.pyplot as plt  
from ipywidgets import *  
from IPython.display import display  
from IPython.html import widgets  
plt.style.use('ggplot')

NUMBER_OF_PINGS = 4

#displaying the text widget
text = widgets.Text(description="Domain to ping", width=200)  
display(text)

#preparing the plot 
data = pd.DataFrame()  
x = range(1,NUMBER_OF_PINGS+1)  
plots = dict()  
fig, ax = plt.subplots()  
plt.xlabel('iterations')  
plt.ylabel('ms')  
plt.xticks(x)  
plt.show()

#preparing a container to put in created checkbox per domain
checkboxes = []  
cb_container = widgets.HBox()  
display(cb_container)

#add button that updates the graph based on the checkboxes
button = widgets.Button(description="Update the graph")

#function to deal with the added domain name
def handle_submit(sender):  
    #a part of the magic inside python : pinging
    res = !ping -c {NUMBER_OF_PINGS} {text.value}
    hits = res.grep('64 bytes').fields(-2).s.replace("time=","").split()
    if len(hits) == 0:
        print "Domain gave error on pinging"
    else:
         #rebuild plot based on ping result
        data[text.value] = hits
        data[text.value] = data[text.value].astype(float)
        plots[text.value], = ax.plot(x, data[text.value], label=text.value)
        plt.legend()
        plt.draw()
        #add a new checkbox for the new domain
        checkboxes.append(widgets.Checkbox(description = text.value, value=True, width=90))
        cb_container.children=[i for i in checkboxes]
        if len(checkboxes) == 1:
            display(button)

#function to deal with the checkbox update button       
def on_button_clicked(b):  
    for c in cb_container.children:
        if not c.value:
            plots[c.description].set_visible(False)
        else:
            plots[c.description].set_visible(True)
    plt.legend()
    plt.draw()

button.on_click(on_button_clicked)  
text.on_submit(handle_submit)  
plt.show()  

ValueError: Unrecognized backend string "nbagg": valid strings are ['pdf', 'pgf', 'Qt4Agg', 'GTK', 'GTKAgg', 'ps', 'agg', 'cairo', 'MacOSX', 'GTKCairo', 'WXAgg', 'template', 'TkAgg', 'GTK3Cairo', 'GTK3Agg', 'svg', 'WebAgg', 'CocoaAgg', 'emf', 'gdk', 'WX']

In [5]:
def simple_gen():
    yield "Hello"
    yield "World"


gen = simple_gen()
print(next(gen))
print(next(gen))

Hello
World


In [6]:
def coro():
    hello = yield "Hello"
    yield hello
 
 
c = coro()
print(next(c))
print(c.send("World"))

Hello
World
